In [20]:
import praw
import pandas as pd
import unicodedata
from sqlalchemy import create_engine
from sqlalchemy.types import String

In [21]:
USERAGENT = '/u/insight_fu comments'
SUBREDDITS = ["pics", "politics", "leagueoflegends", "GirlGamers"]
DATABASE = 'recentPosts'
MAXPOSTS = 10

In [22]:
r = praw.Reddit(USERAGENT)

Version 2.1.21 of praw is outdated. Version 3.0.0 was released Tuesday June 16, 2015.


In [29]:
multi_reddits = []
for subreddit in SUBREDDITS:
    if subreddit == 'GirlGamers':
        multi_reddits.append(r.get_subreddit(subreddit).get_top_from_day(limit = MAXPOSTS))
    elif subreddit != 'GirlGamers':
        multi_reddits.append(r.get_subreddit(subreddit).get_top_from_hour(limit = MAXPOSTS))

In [30]:
for reddit in multi_reddits:
    print reddit

<generator object get_content at 0x10747caf0>
<generator object get_content at 0x1074c20a0>
<generator object get_content at 0x10717ef50>
<generator object get_content at 0x1074c2190>


In [31]:
subLinks = []
for reddit in multi_reddits:
    for post in reddit:
        subLinks.append({'subreddit': post.subreddit,
                        'postTitle': post.title,
                        'postURL': post.permalink,
                        'postCreated': post.created_utc})

In [32]:
subLinksPD = pd.DataFrame(subLinks)

In [37]:
subLinksPD.head()

,postCreated,postTitle,postURL,subreddit
0,1435291802.0,A flock of lawn flamingos can pick a T-rex cle...,http://www.reddit.com/r/pics/comments/3b5mtg/a...,pics
1,1435291495.0,Can someone tell me exactly where in Weisboden...,http://www.reddit.com/r/pics/comments/3b5m8z/c...,pics
2,1435291950.0,Lightning while flying into Denver Internation...,http://www.reddit.com/r/pics/comments/3b5n33/l...,pics
3,1435290101.0,"My hedgehog went full on Loaf today, who am I ...",http://www.reddit.com/r/pics/comments/3b5jvi/m...,pics
4,1435291697.0,Media v. Reality,http://www.reddit.com/r/pics/comments/3b5mm0/m...,pics


In [33]:
subLinksPD.iloc[0].map(type)

postCreated              <type 'numpy.float64'>
postTitle                      <type 'unicode'>
postURL                        <type 'unicode'>
subreddit      <class 'praw.objects.Subreddit'>
Name: 0, dtype: object

In [34]:
convertThese = ['postTitle', 'postURL']
for convert in convertThese:
    subLinksPD[convert] = subLinksPD[convert].map(lambda x:unicodedata.normalize('NFKD', x).encode('ascii', 'ignore'))

In [35]:
convertThese = ['subreddit', 'postCreated']
for convert in convertThese:
    subLinksPD[convert] = subLinksPD[convert].map(str)

In [36]:
engine = create_engine("mysql+pymysql://root@localhost/" + str(DATABASE))
subLinksPD.to_sql('recent', engine, if_exists="replace")